<center>
<img src="https://cornell-tech-logos.fvcproductions.com/assets/img/logos/twisted-t/png/red.png" width=100 />
    <h1>ORIE 5380 / CS 5727</h1>
    <h3>Andrea Lodi, Fall 2021</h3>
</center>

In [1]:
#Using smth means: pre-compile and load smth as a package.
using Pkg;
#Set the Gurobi path
ENV["GUROBI_HOME"] = "C:/gurobi912/win64"
#For first timers (comment after): compile Gurobi with the given path
Pkg.build("Gurobi")
#Add the packages that may not be installed.
Pkg.add(["Latexify", "Polyhedra", "Plots", "Makie", "CDDLib", "JuMP", "Gurobi", "AbstractPlotting"])
#Load the packages
using JuMP, Latexify, Gurobi, LinearAlgebra, Polyhedra, CDDLib, Plots, SparseArrays, Makie, AbstractPlotting

display("Initialization done! All set :-)")

    Building Gurobi → `C:\Users\Alexr\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\fd27e4cda2c8aae18257eab0c1969f54010f7df0\build.log`
Precompiling project...
  ✓ Gurobi
  1 dependency successfully precompiled in 21 seconds (216 already precompiled, 1 skipped during auto due to previous errors)
    Updating registry at `C:\Users\Alexr\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Manifest.toml`


"Initialization done! All set :-)"

┌ Warning: Everything from AbstractPlotting.jl got moved to Makie.jl. 
│ Have a look at the README for information on how to upgrade: 
│ https://github.com/JuliaPlots/AbstractPlotting.jl
└ @ AbstractPlotting C:\Users\Alexr\.julia\packages\AbstractPlotting\2A5iv\src\AbstractPlotting.jl:261


# Chapter 9 First LP

In [2]:
#Building the JuMP model
using JuMP, Gurobi

#Initialize the model with a given optimizer, say Gurobi
cloudExample = Model(Gurobi.Optimizer)

#Turn off verbose output
set_optimizer_attribute(cloudExample, "OutputFlag", 0)

@variable(cloudExample, xm, integer = false, base_name = "xm", lower_bound = 0.0) 
#@variable(cloudExample, xm >= 0)

#xs = @variable(cloudExample, base_name = "xs", lower_bound = 0.0, upper_bound = 140.0)
@variable(cloudExample,0 <= xs <= 140)

#Set the objective. The arguments are the model object, the objective sense and its expression.
@objective(cloudExample, Max, 2400 * xm + 3200 * xs)

#Let's build the first constraint with an expression
#Empty expression
constraint_expr = @expression(cloudExample, 0)
constraint_expr = constraint_expr +  100 * xm
constraint_expr += 40 * xs

#Add the constraint from the expression. The arguments are model, expression LHS, sense and RHS
@constraint(cloudExample, firstConst, constraint_expr <= 10000)

#Let's build the second constraint directly as a constraint
@constraint(cloudExample, secondConst, 200 * xm + 400 * xs <= 60000)

#Let's print the model
display(latex_formulation(cloudExample))

#Let's write the model to file
write_to_file(cloudExample, "CloudExample.lp")

$$ \begin{aligned}
\max\quad & 2400 xm + 3200 xs\\
\text{Subject to} \quad & 100 xm + 40 xs \leq 10000.0\\
 & 200 xm + 400 xs \leq 60000.0\\
 & xm \geq 0.0\\
 & xs \geq 0.0\\
 & xs \leq 140.0\\
\end{aligned} $$

Academic license - for non-commercial use only


In [3]:
#optimize the model
optimize!(cloudExample)

#get the status of the optimization
#either
println("Status = ", raw_status(cloudExample))
#or 
ts = termination_status(cloudExample)
println("Status = $ts")

#print the solution: value and variables
println("Optimal Objective Function value: ", objective_value(cloudExample))
println("Optimal Solutions:")
println("xm = ", value(xm))
println("xs = ", value(xs))


Academic license - for non-commercial use only
Status = Model was solved to optimality (subject to tolerances), and an optimal solution is available.
Status = OPTIMAL
Optimal Objective Function value: 520000.0
Optimal Solutions:
xm = 50.0
xs = 125.0


# Chapter 9 Second LP

In [4]:
#Problem data
#We initialize the optimization's problem data

C=[2.0 4.0 5.0;
   3.0 6.0 8.0; 
   8.0 4.0 9.0]::Matrix{Float64}

#get the size of the matrix
noTechs = size(C,1)
noJobs = size(C,2)

#print some statistics
print("There are ", noTechs, " technicians and ", noJobs, " jobs.")

There are 3 technicians and 3 jobs.

In [5]:
#Building the JuMP model
using JuMP, Gurobi

#Initialize the model with a given optimizer, say Gurobi
assignmentExample = Model(Gurobi.Optimizer)

#set_optimizer_attribute(assignmentExample, "OutputFlag", 0)

#create the matrix of variables in compact form
@variable(assignmentExample, x[1:noTechs,1:noJobs], base_name = "x", lower_bound = 0.0)

#Set the objective. The arguments are the model object, the objective sense and its expression.
#@objective(assignmentExample, Max, dot(C, x))
@objective(assignmentExample, Max, sum(sum(C[i,j] * x[i,j] for i in 1:noTechs) for j in 1:noJobs))

#set constraints for the technicians
for i = 1:noTechs
    constraint_expr = @expression(assignmentExample, 0)
    for j = 1:noJobs
        constraint_expr += x[i,j]
    end
    #Add the constraint from the expression. The arguments are model, expression LHS, sense and RHS
    @constraint(assignmentExample, constraint_expr == 1)
end

#set constraints for the jobs
for j = 1:noJobs
    @constraint(assignmentExample, sum(x[i,j] for i in 1:noTechs) == 1)
end

#Let's print the model
display(latex_formulation(assignmentExample))

#Let's write the model to file
write_to_file(assignmentExample, "assignmentExample.lp")
    

$$ \begin{aligned}
\max\quad & 2 x_{1,1} + 3 x_{2,1} + 8 x_{3,1} + 4 x_{1,2} + 6 x_{2,2} + 4 x_{3,2} + 5 x_{1,3} + 8 x_{2,3} + 9 x_{3,3}\\
\text{Subject to} \quad & x_{1,1} + x_{1,2} + x_{1,3} = 1.0\\
 & x_{2,1} + x_{2,2} + x_{2,3} = 1.0\\
 & x_{3,1} + x_{3,2} + x_{3,3} = 1.0\\
 & x_{1,1} + x_{2,1} + x_{3,1} = 1.0\\
 & x_{1,2} + x_{2,2} + x_{3,2} = 1.0\\
 & x_{1,3} + x_{2,3} + x_{3,3} = 1.0\\
 & x_{1,1} \geq 0.0\\
 & x_{2,1} \geq 0.0\\
 & x_{3,1} \geq 0.0\\
 & x_{1,2} \geq 0.0\\
 & x_{2,2} \geq 0.0\\
 & x_{3,2} \geq 0.0\\
 & x_{1,3} \geq 0.0\\
 & x_{2,3} \geq 0.0\\
 & x_{3,3} \geq 0.0\\
\end{aligned} $$

Academic license - for non-commercial use only


In [6]:
#optimize the model
optimize!(assignmentExample)

#get the status of the optimization
#either
println("Status = ", raw_status(assignmentExample))
#or 
ts = termination_status(assignmentExample)
println("Status = $ts")

#print the solution: value and variables
println("Optimal Objective Function value: ", objective_value(assignmentExample))
println("Optimal Solutions:")
println("x = ",value.(x))

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x62fe8bb3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+31   1.800000e+31   4.900000e+01      0s
       5    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds
Optimal objective  2.000000000e+01
Status = Model was solved to optimality (subject to tolerances), and an optimal solution is available.
Status = OPTIMAL
Optimal Objective Function value: 20.0
Optimal Solutions:
x = [0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0]
